In [3]:
import pandas as pd
import numpy as np
import requests

# input
DATA_FOLDER = './data/'
MOVIES_FOLDER = DATA_FOLDER + 'movies_summaries/'
CHARACTERS_FILE = MOVIES_FOLDER + 'character.metadata.tsv'

# output
ETHNICITY_FILE = './gen/ethnicities.tsv'

# import characters dataset
character_columns = ['wiki_movie_id', 'freebase_movie_id', 'm_release_date', 'name', 'a_dob', 'a_gender', 'a_height', 'a_ethnicity_freebase_id', 'a_name', 'a_age_at_release', 'freebase_char/a_map', 'freebase_char_id', 'freebase_a_id']
characters = pd.read_csv(CHARACTERS_FILE, sep='\t', names=character_columns, index_col=False)

In [4]:
def get_ethnicities(ethnicities_id):    
    """
        Recover ethnicity names from freebase id's using sparql query

        :enthicities_id: list of freebase id's
    """
    url = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"

    query = '''
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wikibase: <http://wikiba.se/ontology#>

        SELECT  ?s ?sLabel ?p  ?o ?oLabel WHERE {{
            {}

            SERVICE wikibase:label {{
                bd:serviceParam wikibase:language "en" .
            }}
        }}
    '''

    # add every id in UNION
    ethnicities_id = ethnicities_id.apply(lambda x: '{{?s wdt:P646 "{}"}}'.format(x))
    ethnicities_id.iloc[1:] = ethnicities_id.iloc[1:].apply(lambda x: ' UNION ' + x)

    query = query.format(ethnicities_id.str.cat())
    response = requests.get(url, params = {'format': 'json', 'query': query})
    return response

# get all unique ethnicity id's
ethnicities_id = pd.Series(characters['a_ethnicity_freebase_id'].unique())
ethnicities_id = ethnicities_id.iloc[1:] # remove nan

# We cannot query all id's at once. We need to split the query in multiple requests.
ethnicities = []
idx_ranges = np.linspace(0, ethnicities_id.size, 5, dtype=int)

for i in range(1, len(idx_ranges)):
    idx_range = np.arange(idx_ranges[i-1], idx_ranges[i])
    response = get_ethnicities(ethnicities_id.iloc[idx_range])

    # add all results to the list
    results = response.json()
    for res in results['results']['bindings']:
        ethnicities.append(res['sLabel']['value'])

# values which don't have information
undefined_values = ['Q31340083', 'Q97377726', 'Q54864438', 'Q56408633', 'Q25467191']

with open(ETHNICITY_FILE, 'w', encoding='utf-8') as f:
    for ethnicity in ethnicities:
        if ethnicity not in undefined_values:
            f.write("{}\n".format(ethnicity))

print("Data generated in {}".format(ETHNICITY_FILE))

Data generated in ./gen/ethnicities.txt


In [5]:
import time

def get_one_ethnicity(ethnicity_id):    
    """
        Recover ethnicity names from freebase id's using sparql query

        :enthicities_id: list of freebase id's
    """
    url = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"

    query = '''
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wikibase: <http://wikiba.se/ontology#>

        SELECT  ?s ?sLabel ?p  ?o ?oLabel WHERE {{
            {}

            SERVICE wikibase:label {{
                bd:serviceParam wikibase:language "en" .
            }}
        }}
    '''

    # add every id in UNION
    ethnicity_id = '{{?s wdt:P646 "{}"}}'.format(ethnicity_id)
    query = query.format(ethnicity_id)

    response = requests.get(url, params = {'format': 'json', 'query': query})
    return response

ethnicities = {}
for i, ethnicity_id in enumerate(ethnicities_id):
    response = get_one_ethnicity(ethnicity_id)
    if response.status_code == 200:
        result = response.json()
        if len(result['results']['bindings']) > 0:
            ethnicities[ethnicity_id] = result['results']['bindings'][0]['sLabel']['value']
        else:
            print("No matching found for {}".format(ethnicity_id))
    else:
        print("Error: {}".format(response.status_code))
        print("Line: {}".format(i))

    if i % 10 == 0:
        print("Sleeping for 10 seconds...")
        print("Elements treated: {}".format(i))
        time.sleep(10)

No matching found for /m/044038p
Sleeping for 10 seconds...
Elements treated: 0
Sleeping for 10 seconds...
Elements treated: 10
No matching found for /m/02p1pl6
No matching found for /m/0bjbszh
No matching found for /m/075dhf0
Sleeping for 10 seconds...
Elements treated: 20
Sleeping for 10 seconds...
Elements treated: 30
No matching found for /m/092h2qt
Sleeping for 10 seconds...
Elements treated: 40
Sleeping for 10 seconds...
Elements treated: 50
No matching found for /m/04kbvpz
No matching found for /m/03ftx7
Sleeping for 10 seconds...
Elements treated: 60
No matching found for /m/0747611
No matching found for /m/047l_90
Sleeping for 10 seconds...
Elements treated: 70
Sleeping for 10 seconds...
Elements treated: 80
Sleeping for 10 seconds...
Elements treated: 90
Sleeping for 10 seconds...
Elements treated: 100
No matching found for /m/0283js_
Sleeping for 10 seconds...
Elements treated: 110
Sleeping for 10 seconds...
Elements treated: 120
Sleeping for 10 seconds...
Elements treated: 

In [ ]:
ethnicities

In [ ]:
with open(ETHNICITY_FILE, 'w', encoding='utf-8') as f:
    for freebase_id, ethnicity in ethnicities.items():
        if ethnicity not in undefined_values:
            f.write("{}\t{}\n".format(freebase_id, ethnicity))